05.06.2019

Image Processing Physics

Julia Herzen, Klaus Achterhold, (Lorenz Hehn, Sebastian Allner)

#  Segmentation: Counting stars

Today's question:
* How many stars are there in a particular image of the night sky?
* What are their apparent sizes?

Applied methods:
* Binary thresholding
* Morphological operations (opening, closing)
* Labeling and analysis of labeling results

## a) Preliminaries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.ndimage as nd

In [ ]:
plt.rcParams['figure.figsize'] = (14, 8)

Load the stars image:

In [ ]:
img = plt.imread('stars.jpg')

* Sum up all color channels to get a grayscale image.
* Sum or average along axis 2
* Rescale the final image to [0.0, 1.0]

In [ ]:
img = ??

**Expert question:** Compare the data types of ``img`` and the version of ``img`` that was summed-up along axis 2.

(Determine the datatype of array ``a`` with ``a.dtype()``).

* What does the output mean?
* Why is it necessary that a different data type is used for the summed-up array? What could happen if the same data type was used?
* Why is it also reasonable to use a different data type when ``np.mean`` is applied instead of ``np.sum``?

Look at the normalized image using ``plt.imshow``. Use the ``vmin`` and ``vmax`` parameters!

In [ ]:
plt.figure(1)
plt.title('Original image')
plt.imshow(img, cmap='gray', interpolation='none', vmin=??, vmax=??)
plt.colorbar()

## b) Thresholding

Determine a threshold value that separates the stars from the background.

Apply binary thresholding with different values until you find a value that separates all stars from the background.



**Note:** Set the sign such that the stars have the value ``True`` or ``1``.
This is important for the order of applied morphological operations:
If the thresholding is such that the star areas are ``False``, the order of
binary closing and opening must be interchanged!

In [ ]:
threshold = ??
img_thr = ??

In [ ]:
plt.figure(2)
plt.title('Binary thresholding')
plt.imshow(img_thr, cmap='gray', interpolation='none')

## c) Morphological operations

**Apply opening and closing** to this binary image to join "fuzzy" areas around stars:

### c.1) Closing

"Closing" is supposed to mean that "holes" (pixels with value 0) are closed.

In other words, the "non-holes" (regions with values 1) grow in size by
applying the kernel on each pixel with value 1.

In [ ]:
kernel_closing = np.array([[??, ??, ??], [??, ??, ??], [??, ??, ??]])
img_thr_cl = nd.binary_closing(img_thr, structure=kernel_closing)

In [ ]:
plt.figure(3)
plt.title('Applied binary closing')
plt.imshow(img_thr_cl, cmap='gray', interpolation='none')

### c.2) Opening

Remove isolated pixels around the moon by opening with an appropriate kernel

(suggestion: A kernel where only two adjacent pixels are set to `1` eliminates single pixels with value 1!)

In [ ]:
kernel_opening = np.array(??)
img_thr_cl_op = nd.binary_opening(img_thr_cl, structure=kernel_opening)

In [ ]:
plt.figure(4)
plt.title('Applied closing, then opening')
plt.imshow(img_thr_cl_op, cmap='gray', interpolation='none')

### c.3) "Fine-tuning"

Play around with all the morphological options in the `scipy.ndimage` (`nd`)
package to increase the quality of the segmentation (if still needed):

In [ ]:
kernel3 = np.array([[???, ???, ???], [???, ???, ???], [???, ???, ???]])
img_final = nd.binary_???(img_thr_cl_op, structure=kernel3)
# ... and potentially additional operations

In [ ]:
plt.figure(5)
plt.title('Final segmentation')
plt.imshow(img_final, cmap='gray', interpolation='none')

## d) Check if you lost any stars!

There are 2 possible issues:
1. Stars were not recognized (false negatives)
2. Noise or artifacts were incorrectly identified as stars (false positives)

Checking for case 1 can e.g. be done by applying the inverted star mask to the original image.
Any bright spots in the resulting image may indicate a failure of masking that particular star.

Case 2 works exactly the other way around: masking the original image with the determined star
mask and identifying anything that doesn't look like a star. It helps to compare it with the
original, unmasked image, and use a lower values for `vmax` so that noise is more apparent.

You may have to zoom in for details, hence we change the backend to use "regular"
in individual windows.

If you get a lot of false negatives or positives, modify the parameters in c)!

In [ ]:
#%matplotlib notebook  # you can try this too, but it's not as great...
%matplotlib qt

In [ ]:
opts = dict(cmap='gray', vmin=0.)
f, [ax1, ax2, ax3] = plt.subplots(1, 3, sharex=True, sharey=True, figsize=(10,5))
ax1.imshow(~img_thr_cl_op * img, vmax=1., **opts)
ax1.set_title('False negatives:\nUndetected stars show up\nhere as bright spots')

ax2.imshow(img_thr_cl_op, vmax=.1, **opts)
ax2.set_title('Only real stars (no noise)\nshould show up here')

ax3.imshow(img, vmax=.1, **opts)
ax3.set_title('Original image')

f.tight_layout()

## e) Labeling and analysis of results
Once you're done, **label your image** with ``nd.label``! This "labels" contiguous areas in a binary image with value 1 (i.e., the individual masked stars). The output of the function is an image where every pixel in a contiguous region is assigned a serial number.

Go back to inline plotting:

In [ ]:
%matplotlib inline
plt.rcParams['figure.figsize'] = (14, 8)

In [ ]:
img_lbld, N = nd.label(??)

Question: **What is `N`**? Check with the documentation of `nd.label`!

In [ ]:
plt.figure(7)
plt.imshow(img_lbld, cmap='jet', interpolation='none')
plt.colorbar()

Use nd.find_objects to return a list of slices through the image for each star:

In [ ]:
slice_list = nd.find_objects(img_lbld)

Note: A **slice object** is a slicing operation which exists
independently of an array. So instead of writing

    B = A[100:200, 100:200]
    
you could also write:

    s = (slice(100, 200, None), slice(100, 200, None))
    B = A[s]

or even shorter:

    s = np.s_[100:200, 100:200]
    B = A[s]

One advantage is of course that you could apply identical
slicing to different arrays:

    s = np.s_[100:200, 100:200]
    B = A[s]
    C = D[s]

For a labelled image the function `nd.find_objects` returns a list of such slices `s`,
the n-th slice defining rectangle around the n-th labelled object (i.e., the n-th star).

In [ ]:
starnum = ??

plt.figure(8)
plt.title("Star %i" % starnum)
plt.imshow(img[slice_list[starnum]], cmap='gray', interpolation='none')

Remaining task: Sum up each individual star to get a list of star sizes and
make a detailed histogram (>100 bins). Take care to exclude the moon! This
can be done by sorting the star sizes list and removing the last element

Remember: The slice object ``slice_list[<number>]`` selects one star (either
from the original `img` or the segmented image `img_final`).
Create a list of boolean star images (star_list).
Afterwards, sum their extent up (take care about the datatypes)
to get their sizes and sort the list.

In [ ]:
star_list = [?? for slc in slice_list]
mass_list = [?? for star in star_list]
mass_list_sorted = ??
mass_list_without_moon = ??

Note: List comprehensions are a shorter way of writing for loops.

    output = [do_thing(x) for x in list]

does the same as:

    output = []
    for x in list:
        output.append(do_thing(x))

In [ ]:
plt.figure(9)
plt.title("Sizes of stars")
plt.hist(??);